In [119]:
import pandas as pd
path = '../dataset/'
prefix = "Fecom Inc"

customers_df = pd.read_csv(f'{path}/{prefix} Customer List.csv', sep=';')
geolocations_df = pd.read_csv(f'{path}/{prefix} Geolocations.csv', sep=';')
order_items_df = pd.read_csv(f'{path}/{prefix} Order Items.csv', sep=';')
order_payments_df = pd.read_csv(f'{path}/{prefix} Order Payments.csv', sep=';')
orders_df = pd.read_csv(f'{path}/{prefix} Orders.csv', sep=';')
products_df = pd.read_csv(f'{path}/{prefix} Products.csv', sep=';')
sellers_df = pd.read_csv(f'{path}/{prefix} Sellers List.csv', sep=';')
reviews_df = pd.read_csv(f'{path}/Fecom_Inc_Order_Reviews_No_Emojis.csv', sep=';')



C:\Users\Familia\AppData\Local\Temp\ipykernel_11280\639127878.py:6: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  geolocations_df = pd.read_csv(f'{path}/{prefix} Geolocations.csv', sep=';')


Informacion Vendedores

In [120]:
orders_df['Order_Purchase_Timestamp'] = pd.to_datetime(orders_df['Order_Purchase_Timestamp'])
orders_df['Order_Delivered_Customer_Date'] = pd.to_datetime(orders_df['Order_Delivered_Customer_Date'])
orders_df['Order_Estimated_Delivery_Date'] = pd.to_datetime(orders_df['Order_Estimated_Delivery_Date'])



In [121]:
orders_df['Atraso'] = (orders_df['Order_Delivered_Customer_Date'] - orders_df['Order_Estimated_Delivery_Date']).dt.days
orders_df['Tiempo total'] = (orders_df['Order_Delivered_Customer_Date'] - orders_df['Order_Purchase_Timestamp']).dt.days

In [122]:
items_orders_df = pd.merge(order_items_df, orders_df[['Order_ID', 'Atraso', 'Tiempo total']], on='Order_ID')

In [123]:
reviews_orders_df = pd.merge(items_orders_df , reviews_df[['Order_ID', 'Review_Score']], on='Order_ID')

In [145]:
df = reviews_orders_df.groupby('Seller_ID')
promerdio_review = df['Review_Score'].mean()
total_orders = df['Order_ID'].count()
Atraso_totales = df['Atraso'].apply(lambda x: (x > 0).sum())
review_malas = df['Review_Score'].apply(lambda x: (x <= 2).sum())

seller_stats = pd.DataFrame({
    'promerdio_review': promerdio_review,
    'total_orders': total_orders,
    'Atraso_totales': Atraso_totales,
    'catidad_review_malas': review_malas
}).reset_index()

In [147]:
seller_stats = pd.merge(seller_stats, sellers_df[['Seller_ID', 'Seller_Name']], on='Seller_ID', how='right' )

In [148]:
seller_stats

,Seller_ID,promerdio_review,total_orders,Atraso_totales,catidad_review_malas,Seller_Name
0,d1b65fc7debc3361ea86b5f14c68d2e2,4.560976,41.0,1.0,2.0,NeuroLabsX
1,51a04a8a6bdcb23deccc82b0b80742cf,1.000000,1.0,1.0,1.0,SwiftLabs
2,e49c26c3edfa46d227d5121a6b6e4d37,4.416667,36.0,0.0,3.0,EcoFutures
3,1b938a7ec6ac5061a66a3766e0e75f90,4.242424,33.0,0.0,4.0,HyperHub
4,a7a9b880c49781da66651ccf4ba9ac38,3.750000,4.0,0.0,0.0,EliteAI
...,...,...,...,...,...,...
3090,443d880f15cbd3572885e1d44bf2c478,2.000000,2.0,0.0,2.0,CrimsonPulse
3091,4a3ccda38b2129705f3fb522db62ca31,3.734177,79.0,5.0,19.0,EchoGlade
3092,6c77cd583b36a13aacccf2b3ee23e233,5.000000,1.0,0.0,0.0,FrostHaven
3093,82bd0703a4aefd6b599e5bfdaed378fb,3.600000,25.0,1.0,7.0,LunarRidge


Posible solucion con productos 

In [ ]:
items_reviews = pd.merge(reviews_df, order_items_df, on='Order_ID')

# Unir con productos para obtener características
df = pd.merge(items_reviews, products_df, on='Product_ID')

# Mostrar preview
df.head()

In [7]:
items_reviews = pd.merge(reviews_df, order_items_df, on='Order_ID')

# Unir con productos para obtener características
df = pd.merge(items_reviews, products_df, on='Product_ID')

# Mostrar preview
df.head()

,Review_ID,Order_ID,Review_Score,Review_Comment_Title_En,Review_Comment_Message_En,Review_Creation_Date,Review_Answer_Timestamp,Order_Item_ID,Product_ID,Seller_ID,Shipping_Limit_Date,Price,Freight_Value,Product_Category_Name,Product_Weight_Gr,Product_Length_Cm,Product_Height_Cm,Product_Width_Cm
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2024-01-18 00:00,2024-01-18 21:46,1,fd25ab760bfbba13c198fa3b4f1a0cd3,6d803cb79cc31c41c4c789a75933b3c7,2024-01-18 15:47,185.00,13.63,Sports_Leisure,1300.0,30.0,30.0,35.0
1,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2024-01-18 00:00,2024-01-18 21:46,2,fd25ab760bfbba13c198fa3b4f1a0cd3,6d803cb79cc31c41c4c789a75933b3c7,2024-01-18 15:47,185.00,13.63,Sports_Leisure,1300.0,30.0,30.0,35.0
2,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2024-03-10 00:00,2024-03-11 03:05,1,be0dbdc3d67d55727a65d4cd696ca73c,8e6d7754bc7e0f22c96d255ebda59eba,2024-03-06 12:48,79.79,8.30,Computers_Accessories,245.0,19.0,14.0,14.0
3,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2024-02-17 00:00,2024-02-18 14:36,1,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2024-02-08 10:31,149.00,45.12,Computers_Accessories,6550.0,20.0,20.0,20.0
4,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,I received it well before the stipulated deadl...,2023-04-21 00:00,2023-04-21 22:02,1,52c80cedd4e90108bf4fa6a206ef6b03,a1043bafd471dff536d0c462352beb48,2023-04-13 17:55,179.99,42.85,Garden_Tools,7650.0,20.0,20.0,20.0


In [8]:
df['calificacion_buena'] = df['Review_Score'].apply(lambda x: 1 if x >= 4 else 0)



# Seleccionar características del producto
features = [
    'Product_Weight_Gr', 'Product_Length_Cm', 'Product_Height_Cm',
    'Product_Width_Cm', 'Product_Category_Name'
]

# Eliminar nulos
df = df[features + ['calificacion_buena']].dropna()



df.head()

,Product_Weight_Gr,Product_Length_Cm,Product_Height_Cm,Product_Width_Cm,Product_Category_Name,calificacion_buena
0,1300.0,30.0,30.0,35.0,Sports_Leisure,1
1,1300.0,30.0,30.0,35.0,Sports_Leisure,1
2,245.0,19.0,14.0,14.0,Computers_Accessories,1
3,6550.0,20.0,20.0,20.0,Computers_Accessories,1
4,7650.0,20.0,20.0,20.0,Garden_Tools,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Separar features y variable objetivo
X = df[[
    'Product_Weight_Gr', 'Product_Length_Cm', 'Product_Height_Cm',
    'Product_Width_Cm'
]]
y = df['calificacion_buena']

# Entrenar/test split
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicciones y evaluación
y_pred = model.predict(X_test)


              precision    recall  f1-score   support

           0       0.45      0.16      0.23      5504
           1       0.77      0.94      0.85     16646

    accuracy                           0.74     22150
   macro avg       0.61      0.55      0.54     22150
weighted avg       0.69      0.74      0.69     22150

